In [23]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [24]:
# Replacing with valid values
ENDPOINT = "https://cloth-defects.cognitiveservices.azure.com/"
ENDPOINT2 = "https://clothdefects-prediction.cognitiveservices.azure.com/"
training_key = "d801b02f2ef5425790ebe32b06f024ee"
prediction_key = "2819396149644c6197cd6526f39d54ba"
prediction_resource_id = "/subscriptions/18ae480a-58ae-4be3-baa4-318cc8b683fb/resourceGroups/CV_PrithviAI/providers/Microsoft.CognitiveServices/accounts/Clothdefects-Prediction"

In [25]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT2, prediction_credentials)

In [26]:
publish_iteration_name = "classifyModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project now")
project_name = uuid.uuid4()
project = trainer.create_project(project_name)

Creating project now


In [27]:
# tags in the project
defect_tag = trainer.create_tag(project.id, "1")
no_defect_tag = trainer.create_tag(project.id, "0")

In [60]:
import pandas as pd

images = pd.read_csv('Train_DefectType_PrithviAI.csv')
images.dropna(inplace=True)
images.shape

(4051, 2)

In [61]:
defect_images_df = images[images['defect_flag'] == 1].reset_index(drop=True)
no_defect_images_df = images[images['defect_flag'] == 0].reset_index(drop=True)

In [62]:
defect_images = defect_images_df.set_index("images id").T.to_dict('list')
no_defect_images = no_defect_images_df.set_index("images id").T.to_dict('list')

<ipython-input-62-61f0b38ae4cd>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  defect_images = defect_images_df.set_index("images id").T.to_dict('list')


In [65]:
base_image_location = "New_folder/Images_Unzipped/New_Images4"

print("Adding images now")

image_list = []

for file in defect_images.keys():
    try:
        with open(os.path.join(base_image_location,"new_"+ file), "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=file, contents=image_contents.read(), tag_ids=[defect_tag.id]))
    except:
        pass


for file in no_defect_images.keys():
    try:
        with open(os.path.join(base_image_location,"new_"+ file), "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=file, contents=image_contents.read(), tag_ids=[no_defect_tag.id]))
    except:
        pass

Adding images now


In [66]:
i = 0
while i <= len(image_list):
    if i + 64 > len(image_list):
        image_list_upload = image_list[i:len(image_list)]
    else:
        image_list_upload = image_list[i:i+64]
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list_upload))
    i += 64

    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)

In [67]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


In [68]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


In [82]:
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT2, prediction_credentials)


image_id = []
defect_flag = []
count = 0
for file in os.listdir("test_images/New_Images"):
    with open(os.path.join("test_images/New_Images", file), "rb") as image_contents:
        results = predictor.classify_image(project.id, "Iteration4", image_contents.read())

    # Display the results.
    tag = 0
    max_prob = 0
    for prediction in results.predictions:
        print("\t" "Image {}  ".format(count) + prediction.tag_name + ": {:.2f}%".format(prediction.probability * 100))
        if prediction.probability >= max_prob:
            tag = prediction.tag_name
            max_prob = prediction.probability
    
    image_id.append(str(file))
    defect_flag.append(tag)
    count += 1

	Image 0  1: 64.22%
	Image 0  0: 35.71%
	Image 1  0: 59.87%
	Image 1  1: 39.73%
	Image 2  0: 98.49%
	Image 2  1: 1.50%
	Image 3  1: 84.93%
	Image 3  0: 15.12%
	Image 4  1: 65.12%
	Image 4  0: 34.54%
	Image 5  1: 51.77%
	Image 5  0: 48.63%
	Image 6  0: 95.13%
	Image 6  1: 4.81%
	Image 7  1: 70.85%
	Image 7  0: 28.99%
	Image 8  1: 93.84%
	Image 8  0: 6.24%
	Image 9  0: 58.41%
	Image 9  1: 41.69%
	Image 10  0: 98.75%
	Image 10  1: 1.24%
	Image 11  1: 72.53%
	Image 11  0: 27.28%
	Image 12  1: 83.27%
	Image 12  0: 16.45%
	Image 13  1: 64.04%
	Image 13  0: 35.80%
	Image 14  0: 88.36%
	Image 14  1: 11.59%
	Image 15  0: 99.53%
	Image 15  1: 0.47%
	Image 16  1: 71.49%
	Image 16  0: 28.04%
	Image 17  1: 79.47%
	Image 17  0: 20.29%
	Image 18  0: 55.77%
	Image 18  1: 43.84%
	Image 19  0: 98.54%
	Image 19  1: 1.49%
	Image 20  0: 57.28%
	Image 20  1: 42.58%
	Image 21  1: 78.72%
	Image 21  0: 20.98%
	Image 22  0: 54.20%
	Image 22  1: 45.91%
	Image 23  0: 94.00%
	Image 23  1: 6.04%
	Image 24  0: 91.23

In [83]:
import pandas as pd

df = pd.DataFrame({"images id":image_id, "defect_flag":defect_flag})
df.to_csv("Final_DefectType_Error303.csv", index=False)